In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, exc
from dotenv import load_dotenv
import os

In [2]:
#Importando planilhas
df_milho = pd.read_excel('Grao.xlsx', sheet_name="Milho")
df_soja = pd.read_excel('Grao.xlsx', sheet_name="Soja")

In [3]:
#Preencher os valores nulos com base no ultimo valor não nulo
df_milho.fillna(method='ffill', inplace=True)
df_soja.fillna(method='ffill', inplace=True)

In [4]:
#Visualização dos dados
display(df_milho.head(10))
display(df_soja.head(10))

,Estado,Cidade,Compra
0,PR,Paranaguá,74.00
1,PR,Campo Mourão,71.00
2,PR,Cascavel,70.00
3,PR,Maringá,71.00
4,PR,Ponta Grossa,72.50
5,PR,Guarapuava,71.00
6,SP,São Paulo,75.50
7,SP,Campinas,75.50
8,SP,Sorocaba,71.50
9,SP,Mogiana,66.32


,Estado,Cidade,Compra
0,PR,Paranaguá,143.75
1,PR,Ponta Grossa,139.50
2,PR,Norte,133.50
3,PR,Oeste,133.00
4,PR,Sudoestes,131.50
5,RS,Rio Grande,147.50
6,RS,Missões,142.00
7,RS,Planalto Central,142.00
8,MT,Cuiabá,126.00
9,MT,Rondonópolis,126.00


In [5]:
# Carrega as variáveis do arquivo .env
load_dotenv()
      
# atribui as variaveis do aquivo
host=os.getenv("DB_HOST")
user=os.getenv("DB_USER")
password=os.getenv("DB_PASSWORD")
database=os.getenv("DB_DATABASE")

#cria a conexao
engine = create_engine(fr'mysql+pymysql://{user}:{password}@{host}/{database}')

In [6]:
#Insere os dados do df_milho na tabela tb_milho no Mysql 
try:
    result = df_milho.to_sql('tb_milho', engine, if_exists='append', index=False )
    print(f'{result} linhas inseridas em tb_milho')

    result = df_soja.to_sql('tb_soja', engine, if_exists='append', index=False )
    print(f'{result} linhas inseridas em tb_soja')
except exc.SQLAlchemyError as error:
    print(f'Erro ao conectar ao banco de dados: {error}')

34 linhas inseridas em tb_milho
33 linhas inseridas em tb_soja


In [17]:
# Conferindo os dados inseridos no DB
try:
    with engine.connect() as conn:
        tb_milho = conn.execute(text("SELECT * FROM tb_milho")).fetchall()
        tb_soja = conn.execute(text("SELECT * FROM tb_soja")).fetchall()
        print('registros em tb_milho:', len(tb_milho))
        print('registros em tb_soja:', len(tb_soja))
except exc.SQLAlchemyError as error:
    print(f'Erro ao conectar ao banco de dados: {error}')

registros em tb_milho: 34
registros em tb_soja: 33
